In [2]:
from  urllib.request import Request,urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from IPython.display import display
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import os
import time
import re
import random

In [3]:
base ="https://www.premierleague.com/stats/top/clubs/{}?se={}"
stats= ["wins","losses","goals","total_yel_card","total_red_card","total_sub_on","total_scoring_att","ontarget_scoring_att","hit_woodwork","att_hd_goal","att_pen_goal","att_freekick_goal","att_ibox_goal","att_obox_goal","goal_fastbreak","total_offside","clean_sheet","goals_conceded","saves","outfielder_block","interception","total_tackle","last_man_tackle","total_clearance","head_clearance","offside_provoked","own_goals","penalty_conceded","pen_goals_conceded","fouls","total_pass","total_through_ball","total_long_balls","backward_pass","total_cross","corner_taken"]
season_map ={"16":"2007","42":"2015","17":"2008",
             "18":"2009","19":"2010","20":"2011",
            "21":"2012","22":"2013","27":"2014",
            "42":"2015"}

#season_map={"16":"2007"}
#stats=["head_clearance","total_tackle","total_long_balls","total_pass","total_cross","backward_pass"]



In [63]:
def getFile(browser,url,stat,se):
    season = season_map[se]
    filename = stat+"_"+season+".csv"
    file = os.path.join("data","csv",filename)
    if os.path.exists(file):
        return
    browser.get(url)
    time.sleep(random.randint(5,10))
    table=browser.find_element_by_tag_name("table")
    #print (table.text)
    f = open(file,'w')
    text = table.text.split("\n")
    new_text=[]
    for line in text:
        matches = re.match("(.*?) (.*) (.*)",line)
        bits =[matches.group(1),matches.group(2),matches.group(3).replace(",","%")]
        csv_line = ",".join(bits)
        new_text.append(csv_line)
    f.write("\n".join(new_text))
    f.close()
    return 

In [133]:
browser = webdriver.Chrome(executable_path="chromedriver.exe")
for s in season_map.keys():
    for ss in stats:     
        url = base.format(ss,s)
        getFile(browser,url,ss,s)
browser.close()
    

In [98]:
#for each season grab the table
def getTables():
    basic="https://www.premierleague.com/tables?se={}"
    browser = webdriver.Chrome(executable_path="chromedriver.exe")
    for s in season_map.keys():
        
        season = season_map[s]
        file = "data/csv/table_"+season+".csv"
        new_url = basic.format(s)
        browser.get(new_url)
        time.sleep(5)
        table = browser.find_element_by_tag_name("table")
        text = table.text.split("\n")
        real_table = text[9:]
        new_text=[]
        new_text.append("Team,Played,Won,Drawn,Lost,GF,GA,GD,Points")
        for line in real_table:
            if len(line)<3:
                continue;
            matches = re.match("(.*) (.*) (.*) (.*) (.*) (.*) (.*) (.*) (.*)",line)
            bits =[]
            for i in range(1,10):
                bits.append(matches.group(i))
            csv_line = ",".join(bits)
            new_text.append(csv_line)
        f = open(file,'w')
        f.write("\n".join(new_text))
        f.close()        
    browser.close()
    
getTables()

In [11]:
#go through year by year and compile each set of stats for the year into a single file
teams = dict()
zz=["2008"]
xx=["goals","wins"]


for year in season_map.values():
    table =pd.read_csv("data/csv/table_"+year+".csv")
    table=table.replace("Bournemouth","AFC Bournemouth").sort_values("Team")
    table.index+=1
    if os.path.exists("data/csv/who_scored_{}.csv".format(year)):
        who_scored = pd.read_csv("data/csv/who_scored_{}.csv".format(year))
    else:
        who_scored =pd.DataFrame(columns=["possession","passSuccess","team"])
    who_scored = who_scored.replace("Bournemouth","AFC Bournemouth").sort_values("team")
    #display(who_scored)
    #display(table)
    final_frame= pd.DataFrame()
    for stat in stats:
        fname = stat+"_"+year+".csv"
        file =  os.path.join("Data","csv",fname)
        statty = pd.read_csv(file,thousands="%").sort_values(["Club"]).reset_index(drop=True)
        if(statty["Rank"][0]=="No"):
            continue
        #print (stat)
        #display(statty.head())
        final_frame["Club"]=statty["Club"]
        final_frame[stat]=statty["Stat"]
    final_frame["Draws"]=38 -final_frame["wins"]-final_frame["losses"]
    final_frame["Points"]=3*final_frame["wins"]+final_frame["Draws"]
    final_frame["Rank"]= table.index
    final_frame["Possession"]= who_scored["possession"]
    final_frame["passSuccess"]= who_scored["passSuccess"]

    #display(final_frame[["Club","Rank","wins"]])
    final_frame = final_frame.sort_values(["Rank"]).fillna(0)
    final_frame.index = np.array(final_frame["Rank"]-1)
    #display(final_frame[["Club","Rank","wins"]])
    final_frame.to_csv("data/final/prem_"+year+".csv")
